In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
import pandas as pd

# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

In [ ]:
import pandas as pd
df = pd.read_csv('train_df.csv', encoding = 'utf-8-sig')


In [ ]:
df['공사종류(대분류)'] = df['공사종류'].str.split(' / ').str[0]
df['공사종류(중분류)'] = df['공사종류'].str.split(' / ').str[1]
df['공종(대분류)'] = df['공종'].str.split(' > ').str[0]
df['공종(중분류)'] = df['공종'].str.split(' > ').str[1]
df['사고객체(대분류)'] = df['사고객체'].str.split(' > ').str[0]
df['사고객체(중분류)'] = df['사고객체'].str.split(' > ').str[1]

In [ ]:
df.value_counts('공종(중분류)')

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

grouped = df.groupby("사고객체(대분류)")
# 결과 저장을 위한 딕셔너리
avg_similarities = {}

# 각 그룹에 대해 평균 코사인 유사도 계산
for category, group in grouped:
    sentences = group['재발방지대책 및 향후조치계획'].tolist()
    
    # 그룹 내 문장이 2개 미만이면 계산할 수 없으므로 0 또는 np.nan 처리
    if len(sentences) < 2:
        avg_similarities[category] = np.nan
        continue
    
    embeddings = model.encode(sentences, convert_to_tensor=False)
    
    sim_matrix = cosine_similarity(embeddings)
    
    # 대각선은 자기 자신과의 유사도로 제외 (k=1부터 시작)
    n = sim_matrix.shape[0]
    upper_tri_indices = np.triu_indices(n, k=1)
    pairwise_sims = sim_matrix[upper_tri_indices]
    
    avg_similarity = np.mean(pairwise_sims)
    avg_similarities[category] = avg_similarity

# 결과 DataFrame 생성
result_df = pd.DataFrame(list(avg_similarities.items()), columns=['공종(중분류)', '평균유사도'])
print(result_df)

In [ ]:
overall_avg = np.nanmean(list(avg_similarities.values()))
print("전체 평균 유사도:", overall_avg)

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

grouped = df.groupby("공사종류(대분류)")
# 결과 저장을 위한 딕셔너리
avg_similarities = {}

# 각 그룹에 대해 평균 코사인 유사도 계산
for category, group in grouped:
    sentences = group['재발방지대책 및 향후조치계획'].tolist()
    
    # 그룹 내 문장이 2개 미만이면 계산할 수 없으므로 0 또는 np.nan 처리
    if len(sentences) < 2:
        avg_similarities[category] = np.nan
        continue
    
    embeddings = model.encode(sentences, convert_to_tensor=False)
    
    sim_matrix = cosine_similarity(embeddings)
    
    # 대각선은 자기 자신과의 유사도로 제외 (k=1부터 시작)
    n = sim_matrix.shape[0]
    upper_tri_indices = np.triu_indices(n, k=1)
    pairwise_sims = sim_matrix[upper_tri_indices]
    
    avg_similarity = np.mean(pairwise_sims)
    avg_similarities[category] = avg_similarity

# 결과 DataFrame 생성
result_df = pd.DataFrame(list(avg_similarities.items()), columns=['공사종류(대분류)', '평균유사도'])
print(result_df)

In [ ]:
overall_avg = np.nanmean(list(avg_similarities.values()))
print("전체 평균 유사도:", overall_avg)